***1. Name Processing
Task:
Split the Name column into two separate fields:
First Name
Last Name
Use the space as a separator for splitting.
Additional Processing:
Remove common prefixes:
Examples: Mr., Mrs., Miss, Dr.
Remove common suffixes:
Examples: Jr., Sr., II, III***

In [51]:
server='DESKTOP-RJ6N7AA\MSSQLSERVER1'
database='python'
username='sa'
password='1234567890'

In [52]:
import urllib
import pandas as pd
ConnectionString = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""
# URL-encode the connection string for SQLAlchemy
params=urllib.parse.quote_plus(ConnectionString)
print(params)

%0A++++DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B%0A++++SERVER%3DDESKTOP-RJ6N7AA%5CMSSQLSERVER1%3B%0A++++DATABASE%3Dpython%3B%0A++++UID%3Dsa%3B%0A++++PWD%3D1234567890%3B%0A++++TrustServerCertificate%3Dyes%3B%0A


In [53]:
from sqlalchemy import create_engine
from sqlalchemy import text
engine=create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

In [ ]:
customer_df=pd.read_sql("select * from customers",con=engine)
transaction_df=pd.read_sql("select * from transactions",con=engine)
order_df=pd.read_sql("select * from orders",con=engine)


In [ ]:
customer_df['name'] = customer_df['name'].str.extract(r'^(?:Mr\.|Mrs\.|Miss|Ms\.|Dr\.|Prof\.)?\s*(.*?)\s*(?:Jr\.|Sr\.|II|III|IV|PhD|MD)?$')[0]


In [64]:
customer_df[['First Name', 'Last Name']] = customer_df['name'].str.split(pat=' ', n=1, expand=True)
cus=customer_df

In [65]:
customer_df.columns

Index(['customer_id', 'name', 'email', 'phone', 'address', 'registration_date',
       'loyalty_status', 'First Name', 'Last Name'],
      dtype='object')

In [ ]:
new_order=['customer_id', 'name', 'First Name', 'Last Name','email', 'phone', 'address', 'registration_date',
       'loyalty_status']
customer_df=customer_df[new_order]
customer_df

***
2. Country Code Extraction and Mobile Number Enrichment
Task:
Extract the country code from the Address field.
Enrichment:
Map the extracted country code to its respective international phone dialing code.
Prepend this dialing code to the customer's Mobile Number to form a complete, internationally formatted phone number.
***

In [71]:
customer_df['country_code']=customer_df['address'].str.slice(-8,-6)
customer_df

C:\Users\Deepak\AppData\Local\Temp\ipykernel_7120\2281051689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_df['country_code']=customer_df['address'].str.slice(-8,-6)


,customer_id,name,First Name,Last Name,email,phone,address,registration_date,loyalty_status,country_code
0,1,Michelle Kidd,Michelle,Kidd,vayala@example.net,+971 6197234258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,AE
1,2,Brad Newton,Brad,Newton,taylorcatherine@example.net,+356 5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,MT
2,3,Larry Torres,Larry,Torres,dsanchez@example.net,+1 8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,UT
3,4,Kimberly Price,Kimberly,Price,jessicaknight@example.com,+1 4232229779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,CA
4,5,Matthew Phillips,Matthew,Phillips,qwilliams@example.com,+1 2207633522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold,OR
...,...,...,...,...,...,...,...,...,...,...
995,996,Jerry Mcdaniel,Jerry,Mcdaniel,walkerlisa@example.net,+1-684 6389899441,"34746 Smith Gateway, New Sarah, AS 12715",10-02-2025,Silver,AS
996,997,Jodi Simpson,Jodi,Simpson,eric24@example.org,+355 4836252940,"2876 Tucker Road Suite 947, North Tommyborough...",18-04-2024,Bronze,AL
997,998,Crystal Brown,Crystal,Brown,pshaffer@example.net,+49 3907473088,"095 Janice Forest Suite 570, Boltonmouth, DE 7...",30-08-2024,Bronze,DE
998,999,Gregory Duarte,Gregory,Duarte,caitlindunlap@example.org,+1 2574098196,"Unit 6377 Box 7662, DPO AP 03300",16-05-2024,Gold,AP


*** 3. Customer Classification
Task:
Add a new column named Customer_Tier based on the following mappings: ***

In [77]:

tier_map = {
    'Gold': 2,
    'Silver': 1,
    'Bronze': 0
}
customer_df['Customer_Tier'] = customer_df['loyalty_status'].map(tier_map)
print(customer_df[['loyalty_status', 'Customer_Tier']].head())
customer_df


  loyalty_status  Customer_Tier
0           Gold              2
1         Silver              1
2         Bronze              0
3           Gold              2
4           Gold              2


C:\Users\Deepak\AppData\Local\Temp\ipykernel_7120\37295677.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_df['Customer_Tier'] = customer_df['loyalty_status'].map(tier_map)


,customer_id,name,First Name,Last Name,email,phone,address,registration_date,loyalty_status,country_code,Customer_Tier
0,1,Michelle Kidd,Michelle,Kidd,vayala@example.net,+971 6197234258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,AE,2
1,2,Brad Newton,Brad,Newton,taylorcatherine@example.net,+356 5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,MT,1
2,3,Larry Torres,Larry,Torres,dsanchez@example.net,+1 8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,UT,0
3,4,Kimberly Price,Kimberly,Price,jessicaknight@example.com,+1 4232229779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,CA,2
4,5,Matthew Phillips,Matthew,Phillips,qwilliams@example.com,+1 2207633522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold,OR,2
...,...,...,...,...,...,...,...,...,...,...,...
995,996,Jerry Mcdaniel,Jerry,Mcdaniel,walkerlisa@example.net,+1-684 6389899441,"34746 Smith Gateway, New Sarah, AS 12715",10-02-2025,Silver,AS,1
996,997,Jodi Simpson,Jodi,Simpson,eric24@example.org,+355 4836252940,"2876 Tucker Road Suite 947, North Tommyborough...",18-04-2024,Bronze,AL,0
997,998,Crystal Brown,Crystal,Brown,pshaffer@example.net,+49 3907473088,"095 Janice Forest Suite 570, Boltonmouth, DE 7...",30-08-2024,Bronze,DE,0
998,999,Gregory Duarte,Gregory,Duarte,caitlindunlap@example.org,+1 2574098196,"Unit 6377 Box 7662, DPO AP 03300",16-05-2024,Gold,AP,2
